In [ ]:
from keras.applications import VGG16
from keras import models
from keras import layers

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(80, 80, 3))

conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

In [ ]:
!mkdir data

In [ ]:
!tar -xopf data-collection.tar -C data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'data/data-collection/train',
        target_size=(80, 80),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'data/data-collection/validation', 
        target_size=(80, 80),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
          train_generator,
          steps_per_epoch=324//batch_size,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=80//batch_size)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.grid(True)
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from matplotlib import image
from matplotlib import pyplot
import numpy as np
import glob

images = glob.glob('/content/data/data-collection/test/Open/*.jpg') + \
          glob.glob('/content/data/data-collection/test/Closed/*.jpg')

for i in range(0, len(images)):
  img_ = image.imread(images[i])
  img = img_ / 255
  img = img[np.newaxis, ...]

  ret = model.predict(img)
  print("Class 1, Class 2: ", ret[0][0], ret[0][1])

  pyplot.imshow(img_)
  pyplot.show()